In [1]:
import json
import pandas as pd
import requests
import sqlite3 as db
from sqlite3 import Error
from datetime import datetime

%run C:\Users\Joshua\Jupyter_Notebook_Folders\APIkeys.py
%run C:\Users\Joshua\Jupyter_Notebook_Folders\MacroData2\db_functions.py

In [2]:
db_name = 'MacroData2'

In [3]:
def get_BLS_data(seriesIDs, startyear, endyear):
    """
    get data from BLS
    """
    base_url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'  #this will not change
    headers = {'Content-type': 'application/json'}  #This will not changed !

    # For the key seriesid enter a list of series names you wish to download
    # For the key startyear enter the start year inside ""
    # For the key endyear enter the end year inside ""
    
    parameters = { 
        "seriesid":seriesIDs,
        "startyear":str(startyear), 
        "endyear":str(endyear),
        "catalog":True, 
        "calculations":False, 
        "annualaverage":False,
        "aspects":False,
        "registrationkey":os.environ['BLS_API_key'] 
     }

    data = json.dumps(parameters) # Converts the Python dictionary to JSON

    p = requests.post(base_url, data=data, headers=headers)
    json_data = json.loads(p.text)
    
    message = ""
    if json_data['message']:
        #message = "For series " + seriesIDs + ", no data for years: "
        for i in range(len(json_data['message'])):
            message += json_data['message'][i][-4:] + ", "
    
    return message, json_data 

In [4]:
def create_year_increments(startyear, endyear):
    """
    Create start and endyear increments of 20 years or 
    """
    if endyear - startyear < 20:
        return startyear, endyear
    else:
        i = startyear
        increments = []
        while i <= endyear: 
            start_end_array = []
            start_end_array.append(i)
            if i + 19 > endyear:
                start_end_array.append(endyear)
            else:
                start_end_array.append(i + 19)
            increments.append(start_end_array)
            i += 20
    
    return increments

In [5]:
def create_dataframes(json_data):
    for i in range(len(json_data['Results']['series'])):
        df_dict = pd.DataFrame()
        df_dict['series_title'] = json_data['Results']['series'][i]['catalog']['series_title'],
        df_dict['series_id'] = json_data['Results']['series'][i]['catalog']['series_id'],
        df_dict['seasonality'] = json_data['Results']['series'][i]['catalog']['seasonality'],
        df_dict['survey_long_name'] = json_data['Results']['series'][i]['catalog']['survey_name'],
        df_dict['survey_short_name'] = json_data['Results']['series'][i]['catalog']['survey_name'][-6:-1],
        df_dict['survey_abbreviation'] = json_data['Results']['series'][i]['catalog']['survey_abbreviation'],
        df_dict['measure_data_type'] = json_data['Results']['series'][i]['catalog']['measure_data_type'],
        df_dict['area'] = json_data['Results']['series'][i]['catalog']['area'],
        df_dict['item'] = json_data['Results']['series'][i]['catalog']['item']
        df_dict = df_dict.set_index(df_dict['series_id'])
        df_dict = df_dict.drop(columns=['series_id'])

        df = pd.DataFrame(json_data['Results']['series'][i]['data'])
        '''
        df['series'] = json_data['Results']['series'][i]['catalog']['series_id']
        df['month'] = df['period'].str.replace('M', '')
        df['date'] = df['year'] + df['month']
        df = df.sort_values(by=['date'], ignore_index=True)
        df['date'] = pd.to_numeric(df['date'])
        df = df.set_index(df["date"])
        df['latest'] = 0
        #df['latest'].iloc[-1] = 0#df['latest'].iloc[-1].upper()
        df = df.drop(columns=['year', 'period', 'periodName', 'footnotes', 'month', 'date'])

        series_id = json_data['Results']['series'][i]['catalog']['series_id']
        create_data_table(series_id, db_name)
        add_data_to_db(df, series_id, db_name)

        series_dict = series_id + "_dict"
        create_dict_table(series_dict, db_name)
        add_data_to_db(df_dict, series_dict, db_name)
        '''

In [6]:
seriesIDs = ["CUUR0000SA0", "CUSR0000SA0"] 
year_increments = create_year_increments(1913, 2003)
for j in year_increments:
    message, json_data = get_BLS_data(seriesIDs, j[0], j[1])
    create_dataframes(json_data)

In [7]:
json_data

{'status': 'REQUEST_SUCCEEDED',
 'responseTime': 264,
 'message': [],
 'Results': {'series': [{'seriesID': 'CUUR0000SA0',
    'catalog': {'series_title': 'All items in U.S. city average, all urban consumers, not seasonally adjusted',
     'series_id': 'CUUR0000SA0',
     'seasonality': 'Not Seasonally Adjusted',
     'survey_name': 'Consumer Price Index for All Urban Consumers (CPI-U)',
     'survey_abbreviation': 'CU',
     'measure_data_type': 'All items',
     'area': 'U.S. city average',
     'item': 'All items'},
    'data': [{'year': '2003',
      'period': 'M12',
      'periodName': 'December',
      'value': '184.3',
      'footnotes': [{}]},
     {'year': '2003',
      'period': 'M11',
      'periodName': 'November',
      'value': '184.5',
      'footnotes': [{}]},
     {'year': '2003',
      'period': 'M10',
      'periodName': 'October',
      'value': '185.0',
      'footnotes': [{}]},
     {'year': '2003',
      'period': 'M09',
      'periodName': 'September',
      'val

In [10]:
df

NameError: name 'df' is not defined

In [8]:
# Get number of series
number_of_series = len(json_data['Results']['series'])
number_of_series

2

In [9]:
check_df = table_to_df('CUSR0000SA0', db_name)
check_df

DatabaseError: Execution failed on sql 'SELECT * FROM CUSR0000SA0;': no such table: CUSR0000SA0

In [ ]:
check_df = table_to_df('CUSR0000SA0_dict', db_name)
check_df

In [ ]:
conn = create_connection(db_name)
c = conn.cursor()
    conn_execute(db_name, sql_code1) 
    
    conn.close()

## Validate Data to Put into Correct Table

In [ ]:
 

    
def create_insert_trigger(tbl_name, db_name):
    '''
    Enter data into the log table when enter data into the database where the value does not exist or does not
    match the value that already exists
    '''
    conn = create_connection(db_name)
    c = conn.cursor()
    
    sql_code = """CREATE TRIGGER log_value_after_insert 
                    AFTER INSERT ON """ + seriesID + """ 
                    BEGIN INSERT INTO value_logs(      
                        seriesID,
                        year, 
                        period, 
                        periodName, 
                        value, 
                        date, 
                        date_action,
                        action)
                    VALUES(
                        new.seriesID,
                        new.year,
                        new.period,
                        new.periodName,
                        new.value,
                        new.date,
                        datetime('now','localtime'),
                        'INSERT'    
                    );
                    END;""".format(seriesID)
    
    conn_execute((sql_code)
    
    conn.close()    

def create_update_trigger(tbl_name, db_name):
    '''
    Enter data into the log table when enter data into the database where the value does not exist or does not
    match the value that already exists
    '''
    conn = create_connection(db_name)
    c = conn.cursor()
    
    sql_code = """CREATE TRIGGER log_value_after_update 
                    AFTER UPDATE ON """ + seriesID + """
                    WHEN old.value <> new.value
                    BEGIN INSERT INTO value_logs(      
                        seriesID,
                        year, 
                        period, 
                        periodName, 
                        value, 
                        date, 
                        date_action,
                        action)
                    VALUES(
                        new.seriesID,
                        new.year,
                        new.period,
                        new.periodName,
                        new.value,
                        new.date,
                        datetime('now','localtime'),
                        'INSERT'    
                    );
                    END;""".format(seriesID)
    conn_execute((sql_code)
    
    conn.close()

'''def unique_index(tbl_name, db_name)
    """
    Create a unique index for the date value of the table
    """
    conn = create_connection(db_name)
    c = conn.cursor()
    
    sql_code = 'CREATE UNIQUE INDEX idx_date ON ' + tbl_name + '(date)';
    
    conn_execute(sql_code)'''
    
def validate_df(df, tbl_name, db_name):
    """
    check if the data from the api df already exists in the table and 
    if the data in the table is duplicate or inconsistent
    return dataframe df without duplicate data or inconsistent data
    return dataframe df_dup_data with duplicate date
    return dataframe df_inconsistent_data with inconsistent data
    """
    conn = create_connection(db_name)
    c = conn.cursor()
    
    # Set up empty dataframes to use to store data temporarily
    df_not_dup_data = pd.DataFrame(columns = list(df.columns))
    df_dup_data = pd.DataFrame(columns = list(df.columns))
    df_inconsistent_data = pd.DataFrame(columns = list(df.columns))
    
    if check_table_exists(seriesID, db_name) != 0:
        # Check if data already exists in DB
        for i, row in df.iterrows():
            c.execute("SELECT date, value FROM " + tbl_name + " WHERE date = ?;", (df.loc[i]["date"],))
            queryOut = c.fetchall()
            
            # Data does not already exist in the database 
            if len(queryOut) == 0: 
                df_not_dup_data = pd.concat([df_not_dup_data, df.loc[[i]]])
            # Data already exists in the database with duplicate values
            elif queryOut[0][1] == df.loc[i]["value"]: 
                df_dup_data = pd.concat([df_dup_data, df.loc[[i]]])
            # Data already exists in the database but with different values
            elif queryOut[0][1] != df.loc[i]["value"]:
                df_inconsistent_data = pd.concat([df_inconsistent_data, df.loc[[i]]])            
    else:
        df_not_dup_data = df.copy()
    
    conn.close()
    
    df_not_dup_data = df_not_dup_data.sort_values(by=['date'])
    
    return df_not_dup_data, df_dup_data, df_inconsistent_data



    
    
def get_max_index(tbl_name, db_name):
    '''
    Get the maximum index value of the specified tbl_name
    '''
    conn = create_connection(db_name)
    c = conn.cursor()
    try:
        c.execute("SELECT MAX([index]) FROM " + seriesID)
        max_index = c.fetchone()[0]
        return max_index
    except Error as e:
        print(e)
    finally:
        conn.close()
        
def get_max_date(tbl_name, db_name):
    '''
    Get the maximum date from the specified tbl_name
    '''
    conn = create_connection(db_name)
    c = conn.cursor()
    try:
        c.execute("SELECT date FROM " + seriesID + " WHERE [index] = (SELECT MAX([index]) FROM " + seriesID +")")
        max_date = c.fetchone()[0]
        return max_date
    except Error as e:
        print(e)
    finally:
        conn.close()




